In [ ]:
# from sympy import symbols, cos, sin, diff, Function, simplify

# # Define symbols
# theta = symbols('theta')
# v0, b, a, k, e, h, GM, sigma = symbols('v0 b a k e h GM sigma')

# # Define r(theta)
# r = - a*(e**2-1) / (1 + e* cos(theta))

# # Calculate dr/dtheta
# dr_dtheta = diff(r, theta)

# # Define omega(theta) = dtheta/dt
# omega = (v0*b) / r**2

# # Calculate d^2r/dtheta^2
# d2r_dtheta2 = diff(dr_dtheta, theta)

# # Calculate domega/dtheta
# domega_dtheta = diff(omega, theta)

# dr_dt = diff(r,theta)*omega
# # Calculate d^2r/dt^2 using chain rule
# d2r_dt2 = simplify(d2r_dtheta2 * omega**2 + dr_dtheta * domega_dtheta * omega)

# # Calculate d^2theta/dt^2
# # Differentiate omega(theta)
# d2theta_dt2 = simplify(domega_dtheta * omega)

# # Display the results
# # print(f"omega= {omega}")
# print(f"r = {r}")
# print(f"dr_dt= {dr_dt}")
# # print(f"dr_dtheta = {dr_dtheta}")
# # print(f"d2r_dtheta2 = {d2r_dtheta2}")
# # print(f"domega_dtheta = {domega_dtheta}")
# print(f"d2r_dt2 = {d2r_dt2}")
# # print(f"d2theta_dt2 = {d2theta_dt2}")


In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy import constants as cc, units as uu


def calc_b(e):
    b = GM/c**2
    return b


# Constants
G = cc.G  # gravitational constant in m^3 kg^-1 s^-2
M = cc.M_sun  # mass of the Sun in kg
c = cc.c.si.value  # speed of light in m/s
GM = (G*M).si.value  # gravitational parameter for the Sun in m^3/s^2

# Initial conditions
m_sun = cc.M_sun.si.value
r_sun = cc.R_sun.si.value
# Constants for the hyperbola
d0 = 10*r_sun
v0 = np.sqrt(2*GM/d0)
e0 = 2.0
a = -GM / (2e4**2)  # Example semi-major axis calculation for hyperbolic trajectory
b = calc_b(e0)
print(b)


def r_derivatives(theta,a0, e0, b0, afactor, bfactor, efactor):
    a= a0*afactor
    e=  e0*efactor
    b = b0*bfactor
    # Expression for r, r_dot, and r_double_dot
    r = -a*(e**2 - 1)/(e*np.cos(theta) + 1)
    dr_dt= -b*e*c*np.sin(theta)/(a*(e**2 - 1))
    d2r_dt2 = -b**2*e*c**2*(e*np.cos(theta) + 1)**2*np.cos(theta)/(a**3*(e**2 - 1)**3)
    return r, dr_dt, d2r_dt2

def calc_r(theta, a, e):
    r = -a*(e**2 - 1)/(e*np.cos(theta) + 1)
    return r


def newtonian_grav_accel(GM, r):
    a_theoretical = -GM / r**2
    return a_theoretical

def new_grav_accel(GM, r, r_dot):
    alpha = np.sqrt(2*GM/(np.abs(r)*c**2)) # alpha is alreayd v/c
    v = alpha*c
    gamma_v = 1 / np.sqrt(1 -alpha**2)
    B = (1 + (gamma_v - 1) * (r_dot / v)**2)
    a_theoretical = -GM / gamma_v / r**2 / B
    return a_theoretical

def error_functionHU(x):
    afactor, bfactor, efactor = x
    theta_inf = np.arccos(-1 / e0)/1.1
    def integrand(theta):
        r, r_dot, r_double_dot = r_derivatives(theta,a, e0, b, afactor, bfactor, efactor)
        a_theoretical = new_grav_accel(GM, r, r_dot)
        a_numerical = r_double_dot - (b*c)**2/r**3 # Assuming r_double_dot is defined to calculate \ddot{r}
        return (a_theoretical - a_numerical)**2

    integral_error, _ = quad(integrand, -theta_inf, theta_inf, epsabs=1e-14, epsrel=1e-14)  # Use the correct limits
    return 1E6*integral_error

def error_functionNewton(x):
    afactor, bfactor, efactor = x
    theta_inf = np.arccos(-1 / e0)/1.1
    def integrand(theta):
        r, r_dot, r_double_dot = r_derivatives(theta,a, e0, b, afactor, bfactor, efactor)
        a_theoretical = newtonian_grav_accel(GM, r)
        a_numerical = r_double_dot -  - (b*c)**2/r**3  # Assuming r_double_dot is defined to calculate \ddot{r}
        return (a_theoretical - a_numerical)**2

    integral_error, _ = quad(integrand, -theta_inf, theta_inf, epsabs=1e-14, epsrel=1e-14)  # Use the correct limits
    return 1E6*integral_error

def optimize_trajectory(errf):
    initial_guess = [1, 1, 1]  # Initial guesses for sigma and e
    result = minimize(errf, initial_guess, method='Nelder-Mead', tol=1e-6)
    afactor_optimized, bfactor_optimized, efactor_optimized = result.x
    optimized_a = a*afactor_optimized
    optimized_b = b*bfactor_optimized
    optimized_e = e0*efactor_optimized
    print("optimized_a =", optimized_a)
    print("optimized_b =", optimized_b)
    print("optimized_e =", optimized_e)

    # Calculate the deflection angle Δθ in radians
    delta_theta = 2 * np.arctan(np.sqrt(optimized_e**2 - 1) / optimized_e)
    delta_theta_degrees = np.degrees(delta_theta)
    print(f"The deflection angle Δθ is {delta_theta:.6}")
    return delta_theta


# optimize_trajectory(error_functionNewton)
# optimize_trajectory(error_functionHU)

In [ ]:
AngleNewton = optimize_trajectory(error_functionNewton)

In [ ]:
optimize_trajectory(error_functionNewton)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

e=e0
# Function to calculate r based on theta
def r(theta, a, e):
    return - a * (e**2 - 1) / (1 + e * np.cos(theta))

# Generate theta values from -theta_inf to theta_inf
theta_inf = np.arccos(-1 / e)/1.1
# p = 20*a
# theta_inf = np.arccos(1-a/p/(e**2-1))
theta_values = np.linspace(-theta_inf, theta_inf, 1000)

# Plotting the hyperbola
plt.figure(figsize=(8, 6))

optimized_a = -431315429999.9988
optimized_b = 147.6625038050113
optimized_e = 3.3999999999999924

r_calc = calc_r(theta_values, optimized_a, e)
x_values = r_calc * np.cos(theta_values)
y_values = r_calc * np.sin(theta_values)

plt.plot(x_values, y_values, label=f'CalcHyperbola: a={optimized_a}, e={optimized_e}')
# Calculate r for each theta
r_values = r(theta_values, a, optimized_e)
# Convert polar coordinates to Cartesian for plotting
x_values = r_values * np.cos(theta_values)
y_values = r_values * np.sin(theta_values)

plt.plot(x_values, y_values, label=f'Hyperbola: a={a}, e={e}')
plt.scatter([0], [0], color='yellow', label='Central Body')  # Central body (e.g., the Sun)
plt.title('Hyperbolic Trajectory around a Central Body')
plt.xlabel('x-coordinate')
plt.ylabel('y-coordinate')
plt.axhline(0, color='grey', linestyle='--')  # Asymptote
plt.axvline(0, color='grey', linestyle='--')  # Asymptote
plt.grid(True)
plt.legend()
plt.axis('equal')  # Ensure equal scaling for x and y axes
plt.show()


In [ ]:
r_values

In [ ]:
delta =1
sigma = 1
efactor =1
mass = 3000

# b = -GM/(v0**2)*(e**2-1)/(e+1)*np.sqrt(e**2-1)

def calc_b(e):
    b = GM/c**2
    return b

b=calc_b(e0)


r1, dr_dt1, d2r_dt21 =r_derivatives(theta_values,a, e0, b, sigma, efactor, delta)
newton_accel = newtonian_grav_accel(GM, r1)
HU_acell = new_grav_accel(GM, r1, dr_dt1)
a_numerical = d2r_dt21 - (b*c**2)/(r1**2)


plt.plot(theta_values, newton_accel-0.00001, color="black")
plt.plot(theta_values, HU_acell+0.00001, color="red")
plt.plot(theta_values, a_numerical, color="green")
plt.show()

In [ ]:
# import numpy as np
# from scipy.optimize import minimize
# from scipy.integrate import quad

# # Constants and initial conditions
# GM = 1.32712440018e20  # Example value for GM
# a = -GM / (2e4**2)  # Example semi-major axis calculation for hyperbolic trajectory
# e = 1.5  # Eccentricity, must be > 1 for hyperbola
# b = 6.685e9  # Impact parameter for example

# def integrand(theta):
#     r = a * (e**2 - 1) / (1 + e * np.cos(theta))
#     theta_dot = (2e4 * b) / r**2
#     dr_dtheta = -a * (e**2 - 1) * e * np.sin(theta) / (1 + e * np.cos(theta))**2
#     d2r_dtheta2 = a * (e**2 - 1) * e * (np.cos(theta) - e * (np.sin(theta)**2 / (1 + e * np.cos(theta)))) / (1 + e * np.cos(theta))**3
#     dr_dt = dr_dtheta * theta_dot
#     d2r_dt2 = (d2r_dtheta2 * theta_dot**2) + (dr_dtheta * -2 * r * theta_dot**3 / (2e4 * b))
#     return (GM / r**2 - (d2r_dt2 + r * theta_dot**2))**2

# theta_inf = np.arccos(-1/e)  # Correct limits calculation
# result, _ = quad(integrand, -theta_inf, theta_inf)
# print(f"Integrated error: {result}")

# initial_guess = [1.0, 1.5]  # Reasonable initial guesses
# opt_result = minimize(lambda x: quad(integrand, -np.arccos(-1/x[1]), np.arccos(-1/x[1]))[0], initial_guess, method='Nelder-Mead')
# print(f"Optimized parameters: {opt_result.x}")
